In [1]:
import sys
sys.path.insert(0, '../BO')

from gp import *
import numpy as np
from sklearn.datasets import fetch_mldata
from LG import *# Load and format the training and testing data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

loss = train_logisitc_regression_(mnist)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# set parameters
bounds = np.array([[-15., 0.0], [5, 200], [20, 2000], [0, 1]])
# maxeps specifies how many BOs we want to run
maxeps = 10
# n_iters is number of iterations we want for each run of BO
n_iters = 100
acqui_eva_num = 5
n_pre_samples = 3
greater_is_better = False
y_list = list()
coor_sigma = 1 * np.array([0.2,0.2,0.2,0.2,0.2,0.2])
burn_in = 10
input_dimension = 4
mode = 'MAP'
acqui_mode = 'MCMC'
acqui_sample_num = 5
process_sample_mode = 'abs'
prior_mode ='exp_prior'
likelihood_mode = 'normal_likelihood'

In [3]:
y_list = list()

for i in range(n_iters):
    y_list.append(list())


for j in range(maxeps):
    print ('Running %d episode' % (j + 1))
    xp, yp = bayesian_optimisation(coor_sigma = coor_sigma, 
                                   burn_in = burn_in, 
                                   input_dimension = input_dimension,
                                   n_iters = n_iters, 
                                   sample_loss = loss, 
                                   bounds = bounds, 
                                   n_pre_samples = n_pre_samples, 
                                   acqui_eva_num = acqui_eva_num,  
                                   greater_is_better=greater_is_better, 
                                   mode = mode, 
                                   acqui_mode = acqui_mode, 
                                   acqui_sample_num = acqui_sample_num,   
                                   process_sample_mode = process_sample_mode, 
                                   prior_mode = prior_mode, 
                                   likelihood_mode = likelihood_mode)
    y_train = yp[n_pre_samples:]
    for idx in range(len(y_train)):
        if idx == 0:
            y_list[idx].append(y_train[idx])
        else:
            y_list[idx].append(np.min(y_train[:idx]))

Running 1 episode
Start presampling...
	Learning rate: 0.165351901942, training epochs: 190, batch size: 274, beta 0.788208798816
		 Accuracy: 0.7366
	Learning rate: 9.16248436286e-06, training epochs: 76, batch size: 407, beta 0.358124901572
		 Accuracy: 0.6977
	Learning rate: 0.00857114050219, training epochs: 105, batch size: 1829, beta 0.599358128344
		 Accuracy: 0.8197
Presampling finished.
	Learning rate: 1.52708894483e-06, training epochs: 178, batch size: 1213, beta 0.742136405712
		 Accuracy: 0.6748
	Learning rate: 0.0142908939349, training epochs: 171, batch size: 1461, beta 0.0909823586856
		 Accuracy: 0.8754
	Learning rate: 4.50198884437e-06, training epochs: 23, batch size: 1208, beta 0.94304639119
		 Accuracy: 0.6718
	Learning rate: 5.93108770415e-07, training epochs: 141, batch size: 1361, beta 0.305288590825
		 Accuracy: 0.6721
	Learning rate: 6.43251166522e-06, training epochs: 189, batch size: 202, beta 0.322778252142


KeyboardInterrupt: 

In [ ]:
mean_list, std_list = return_mean_std(y_list)

In [ ]:
import matplotlib.pyplot as plt
x = [i+1 for i in range(len(y_list))]

plt.errorbar(x, mean_list, yerr = std_list, fmt = '-')

plt.show()

In [ ]:
import pickle
opt_dict = {}
opt_dict['y_list'] = y_list
with open('LR_MCMC_EXP.pkl','wb') as file:
    pickle.dump(opt_dict, file)

In [ ]:
with open('LR_MCMC_EXP.pkl', 'rb') as pickle_file:
    data1 = pickle.load(pickle_file)